In [ ]:
#Choice of parameters

filename = "../230911_data_for_Michael" #xlsx file that contains all data in the different sheets
#extract the data to analyse from the xlsx file
sheetNr = 8
#TrpR: [2,10,12]
#CoCl:  [4,6,8,14,16]
setsUsedForFitting = 3  #   > 0, <  number of concentrations, typically have to kick out some data due to poor behavior (negative concentrations etc),
                        # in particular the low concentration cases

In [ ]:
######################################
######################################
## Choice of sheet
######################################
######################################

from python_scripts.xlsx_file_read_extractor import *
from python_scripts.helper_functions import *
from python_scripts.Fit_functions import *
from python_scripts.Rate_equation_solution import *




used_workbook = Get_workbook_from_xlsx_file(filename, True)
usedSheet = used_workbook[used_workbook.sheetnames[sheetNr]]   #sheet decides the experiment, use 2+ for experiments, 2/3, 4/5 etc are the same, just different channels
dataDict = Get_data_from_sheet(usedSheet) #contains all data in form of dictionary, splitting it up by the used concentrations (and also time extra)
allkeys = list(dataDict.keys()) #keys for the data dict
print('name of the used sheet:')
print(usedSheet.title)
print('found key names:')
for x in allkeys:
    print(x)



In [ ]:
#plot the data to see what it looks like and what has to be done later
#just for visualization, nothing has to be done here
import matplotlib.pyplot as plt

timeKey = allkeys[0]
usedDataKeys = allkeys[setsUsedForFitting:]
if setsUsedForFitting != 1:
    print('Note: not all data is being used')
maxy = 0
timevals = dataDict[timeKey]
yvalslist = []



for x in usedDataKeys:
    yvals = dataDict[x]
    if maxy < max(yvals):
        maxy = max(yvals)
    yvalslist.append(yvals)    
    # plt.plot(timevals,yvals, label=x)

# plt.figure(figsize=[5,3])
fig, ax_dict = plt.subplot_mosaic([['vis','log']],figsize=[9,4])#,
                                #   empty_sentinel="BLANK")


for (x,y) in zip(yvalslist,usedDataKeys):
    ax_dict['vis'].plot(timevals, x, 'x-',markersize=1,lw=1,  label=y)
    ax_dict['log'].plot(timevals, x, 'x-',markersize=1,lw=1,  label=y)

# ax_dict[x].set_title(x + ' fit params')
ax_dict['vis'].set_ylabel('RU')
ax_dict['vis'].set_xlabel(timeKey)
ax_dict['vis'].legend()
ax_dict['log'].set_xlabel(timeKey)
ax_dict['log'].set_yscale('log')
ax_dict['log'].set_title('logscale')
ax_dict['vis'].set_xlim([-1,dataDict[timeKey][-1]])
ax_dict['log'].set_xlim([-1,dataDict[timeKey][-1]])
    
plt.tight_layout()    
plt.suptitle(f'Data of exp. {usedSheet.title}')


# plt.ylim([-2,30])
# plt.savefig('Full_data_exp_' + usedSheet.title + '.pdf' )
# plt.ylim([0,maxy])
# plt.ylim([1e0,1e2])
plt.xlim([-1,dataDict[timeKey][-1]])
# plt.savefig('Data_' + usedSheet.title + ".pdf")

In [ ]:
#plot the data to see what it looks like and what has to be done later

#Choose what is  used in analysis
#Note that fitting is done with the logarithm of the data, so one has to make sure, that  the used data sets are valid (low concentrations have ~0 RU, which can't be fitted)
#to see the problem, one  can plot in yscale (uncomment below statement) and observe the bad behavior of them
timeKey = allkeys[0]
usedDataKeys = allkeys[setsUsedForFitting:]


#######################################################################
#######################################################################
#######################################################################
#important choice of the cutout of the washing peaks, choose such that the behavior is regular
#might be possible to partially automize this, as the washing peaks are very strong, could maybe write a function that tries to do this automatically
startcuts = [2420,3630]  #start cut value of each washing (# elements should be the number of washings)
endcuts = [3570, -1000] #end cut value of each washing, 
#i.e. data[startcuts[0]:endcuts[0]] is the interval that is being used for the 1. washing and so on for the next washings
nrOfDataPoints = 300 #choose how many data points should be used (rate data has too many points, 
#we can strip some data without loss of accuracy, but also increase of fit speed)
#######################################################################
#######################################################################
#######################################################################

yvalsdict = dict() #contains all cutted data sets
t0_estimates = [dataDict[timeKey][x] for x in startcuts] #get the times of the cuts as t0 estimates
timevals = Single_list_cut(dataDict[timeKey], startcuts,endcuts)
list_shrinker(timevals,nrOfDataPoints)
maxy = 0

timevals
for x in usedDataKeys:
    yvals = Single_list_cut(dataDict[x], startcuts,endcuts)
    list_shrinker(yvals,nrOfDataPoints)
    yvalsdict[x] = yvals
    if maxy < max(yvals):
        maxy = max(yvals)
    plt.plot(timevals,yvals, label=x, marker='o',markersize=4)
plt.legend()    
plt.xlabel(timeKey)
plt.ylabel('RU')
plt.title(f'Data of exp. {usedSheet.title}')
# plt.ylim([-2,30])
# plt.savefig('Full_data_exp_' + usedSheet.title + '.pdf' )
# plt.ylim([0,maxy])
#choose to show everything in log scale
# plt.yscale('log')
# plt.ylim([1e0,1e2])
# plt.xlim([0,dataDict[timeKey][-1]])
# plt.savefig('Data_' + usedSheet.title + ".pdf")

In [ ]:
################################################
################################################
#if fitting does not work, might have to adjust fit params here
#visualization might also later change, if other fit params are used
#Last difficult thing to do is the generation of arbitrary figures
#might have to try to generalize this via a function for all data of interest
################################################
################################################

startValDict = dict()
startValDict['k12'] = [[0.3,0],[True,False]]
startValDict['k21'] = [[0.03,0.4],[True,True]]
startValDict['k23'] = [[0.01,0.01],[True,True]]
startValDict['k32'] = [[0.001,0.001],[True,True]]
startValDict['C1_start'] = [70,True]
startValDict['C2_start'] = [0.0001,False]
startValDict['C3_start'] = [0.0001,False]
startValDict['t_0'] = [t0_estimates,[True,True]]
nrWashings = len(startValDict['k12'][0])

boundValDict = dict()
boundValDict['k12'] = [0,1]  #lower and upper values [[lower bounds], [upper bounds]], will partially ignore if false chosen for individual parts
boundValDict['k21'] = [0,1]   #same lower/upper bounds for all params
boundValDict['k23'] = [0,0.2]
boundValDict['k32'] = [0,0.1]
boundValDict['C1_start'] = [2,100] #if no fit, then no keyword stored of it, see C2,C3
boundValDict['t_0'] = [[x - 2 for x in startValDict['t_0'][0]],[x + 2 for x in startValDict['t_0'][0]]]


# Fit_to_function_multiple_washes(timelist,datalist,startValDict,boundValDict,nrWashings)

allResults = dict()
for x in yvalsdict.keys():
    print('fitting for ' + str(x))
    resultfit = Fit_to_function_multiple_washes(timevals,yvalsdict[x],startValDict, boundValDict,nrWashings)
    allResults[x] = resultfit
    plt.plot(timevals,yvalsdict[x], label=x)
    # print(resultfit[0])
    plotOfFit = resultfit[2].call_full_function(timevals,*resultfit[0])
    # print(resultfit[2].fit_positions_and_names())
    plt.plot(timevals,np.array(plotOfFit[1]) + np.array(plotOfFit[2]), '--')
plt.legend()

In [ ]:

#here the rest of plotting all the fitting parameters, for understanding what happened
#also the interesting vals, then finished with my part for fitting all
#probably best to create another class/function that extracts whatever i want from the dict of interest
interestingstuff = [[],[],[]]
xvals = []
for x in allResults.keys():
    xvals.append(x)
    # print(resultfit[2].fit_positions_and_names())
    t0_fitted = [allResults[x][0][-1]]
    t0Concentrations = allResults[x][2].call_full_function(t0_fitted,*allResults[x][0])
    interestingstuff[0].append(t0Concentrations[0][0])
    interestingstuff[1].append(t0Concentrations[1][0])
    interestingstuff[2].append(t0Concentrations[2][0])

plt.plot(xvals, interestingstuff[0], 'X--')
plt.plot(xvals, interestingstuff[1], 'X--')
plt.plot(xvals, interestingstuff[2], 'X--')
plt.xlabel('Motor concentration in nM')
